# GFW Data-API Playground

When developing, to test newly saved code:

- restart kernel
- after each save in the command line use: `!python setup.py develop ` or `!pip install -e .` to install
- then re-import `LMIPy`
- use `py.test` in cli to execute tests

### To release

1. Increment the version in `setup.py`.
2. Push changes to github
3. Checkout to master, pull
3. Remove the `dist` folder
3. In the CLI run `make pypi`
4. Check version on [Colab](https://colab.research.google.com/drive/1tRye2QAytWzuki3DQyg4G7fsFFKlZNic#scrollTo=EN4iMVRm6N0Y)



In [4]:
!python setup.py develop 

from IPython.display import clear_output,display
clear_output()

import LMIPy as lmi
from LMIPy import utils

print(f'LMI ver. {lmi.__version__} ready!')

LMI ver. 0.5.0 ready!


# RW Auth Class

In [6]:
SERVER = 'production'
# SERVER = 'staging'

creds = lmi.Auth(server=SERVER)

## Register new user

In [7]:
email = 'aj.pain89@gmail.com'
name = 'AJ Pain'

creds.register(email=email, name=name)

Registration successful!
We've sent you an email. Click the link in it to confirm your account.


## Login and generate token

Note that generating a token will invalidate existing tokens.

In [7]:
email = 'aj.pain89@gmail.com'
creds.login(email=email)

Password: ········


In [10]:
creds.generateToken()

In [9]:
TOKEN = creds.token

In [11]:
creds.role

'USER'

In [12]:
creds.apps

[]

## Update user

In [8]:
email = 'adam.pain@vizzuality.com'
creds.login(email=email)

Password: ········


In [10]:
print(f"user_id: {creds._id}")
print(f"role: {creds.role}")
print(f"apps: {creds.apps}")

TOKEN = creds.token

user_id: 59db4eace9c1380001d6e4c3
role: ADMIN
apps: ['rw', 'gfw', 'prep', 'aqueduct', 'forest-atlas', 'data4sdgs', 'gfw-climate', 'gfw-pro', 'ghg-gdp', 'ng']


In [17]:
user = creds.getUserByEmail(user_email='aj.pain89@gmail.com', token=TOKEN)
user

https://api.resourcewatch.org/auth/user?app=all&email=aj.pain89@gmail.com


[{'_id': '60eda01babdcd1001b768594',
  'createdAt': '2021-07-13T14:15:55.000Z',
  'email': 'aj.pain89@gmail.com',
  'extraUserData': {'apps': []},
  'id': '60eda01babdcd1001b768594',
  'name': 'AJ Pain',
  'provider': 'local',
  'role': 'USER',
  'updatedAt': '2021-07-13T14:17:35.000Z'}]

In [19]:
uid = user[0]["_id"]
uid

'60eda01babdcd1001b768594'

In [22]:
payload = {
    'extraUserData': {
        'apps': ['gfw']
    },
    'role': 'ADMIN'
}
creds.updateUser(user_id=uid, token=TOKEN, payload=payload)

https://api.resourcewatch.org/auth/user/60eda01babdcd1001b768594


{'_id': '60eda01babdcd1001b768594',
 'createdAt': '2021-07-13T14:15:55.000Z',
 'email': 'aj.pain89@gmail.com',
 'extraUserData': {'apps': ['gfw']},
 'id': '60eda01babdcd1001b768594',
 'name': 'AJ Pain',
 'provider': 'local',
 'role': 'ADMIN',
 'updatedAt': '2021-07-13T14:39:49.000Z'}

# Data API

### Get Data-API Keys

In [5]:
from LMIPy import dataAPI

SERVER = 'production'
# SERVER = 'staging'

In [6]:
## Get data-api creds (staging only; not currently enforced in production)
## Always use Prod RW API TOKEN
creds = dataAPI.Auth(server=SERVER, rw_api_token=TOKEN)

In [7]:
## See keys that I have generated
creds.keys

[GFW Data API Credential: 'AJ Prod Test Key' (does not expire),
 GFW Data API Credential: 'GFW Flagship Production Key' (does not expire),
 GFW Data API Credential: 'GFW Flagship Staging Key' (does not expire),
 GFW Data API Credential: 'AJ data-api' (does not expire)]

In [8]:
## Set data-api key
DATA_API_KEY = creds.keys[1].api_key

### Search Data Catalogue

In [6]:
## Search the catalogue to populate datasets
col = dataAPI.DataCatalogue(server=SERVER, search='umd', token=DATA_API_KEY)
datasets = col.datasets
datasets

NameError: name 'GFWDataset' is not defined

In [10]:
## Select Dataset and populate with metadata as needed
d = datasets[7]
d

GFW Data API Dataset: 'umd_tree_cover_loss'

In [11]:
## Populate metadata etc
d.get_metadata(verbose=True)
d.get_versions(True)
d.get_fields(True)

d.metadata['function']

https://data-api.globalforestwatch.org/dataset/umd_tree_cover_loss/v1.8
https://data-api.globalforestwatch.org/dataset/umd_tree_cover_loss
https://data-api.globalforestwatch.org/dataset/umd_tree_cover_loss/v1.8/fields


'Identifies areas of gross tree cover loss.'

In [12]:
d.metadata

{'added_date': '2014-02-01',
 'analysis': None,
 'cautions': 'In this data set, “tree cover” is defined as all vegetation greater than 5 meters in height, and may take the form of natural forests or plantations across a range of canopy densities. “Loss” indicates the removal or mortality of tree cover and can be due to a variety of factors, including mechanical harvesting, fire, disease, or storm damage. As such, “loss” does not equate to deforestation.\nDue to variation in research methodology and date of content, tree cover, loss, and gain data sets cannot be compared accurately against each other. Accordingly, “net” loss cannot be calculated by subtracting figures for tree cover gain from tree cover loss, and current (post-2000) tree cover cannot be determined by subtracting figures for annual tree cover loss from year 2000 tree cover.\nThe 2011-2019 data was produced using [updated methodology](http://earthenginepartners.appspot.com/science-2013-global-forest/download_v1.7.html). C

### Use Slug Directly

In [8]:
## Instantiate new dataset object using the table name slug directly
dataset_name = 'nasa_viirs_fire_alerts'

d = dataAPI.Dataset(server=SERVER, slug=dataset_name)

In [9]:
## Populate metadata etc
d.get_metadata(verbose=True)
d.get_versions(True)
d.get_fields(True)

d.versions

https://data-api.globalforestwatch.org/dataset/nasa_viirs_fire_alerts/v202105.3
https://data-api.globalforestwatch.org/dataset/nasa_viirs_fire_alerts
https://data-api.globalforestwatch.org/dataset/nasa_viirs_fire_alerts/v202105.3/fields


['v202010', 'v202105.2', 'v202105.3']

In [10]:
# Queryable fields
d.fields

['iso',
 'adm1',
 'adm2',
 'longitude',
 'latitude',
 'alert__date',
 'alert__time_utc',
 'confidence__cat',
 'bright_ti4__K',
 'bright_ti5__K',
 'frp__MW',
 'wdpa_protected_area__iucn_cat',
 'is__umd_regional_primary_forest_2001',
 'is__birdlife_alliance_for_zero_extinction_site',
 'is__birdlife_key_biodiversity_area',
 'is__landmark_land_right',
 'gfw_plantation__type',
 'is__gfw_mining',
 'is__gfw_managed_forest',
 'rspo_oil_palm__certification_status',
 'is__gfw_wood_fiber',
 'is__peatland',
 'is__idn_forest_moratorium',
 'is__gfw_oil_palm',
 'idn_forest_area__type',
 'per_forest_concession__type',
 'is__gfw_oil_gas',
 'is__gmw_mangroves_2016',
 'is__ifl_intact_forest_landscape_2016',
 'bra_biome__name',
 'alert__count',
 'geom',
 'geom_wm']

In [11]:
## Query using a geostore to do zonal stats
gid = 'c3833748f6815d31bad47d47f147c0f0'

In [14]:
sql = """SELECT SUM(gfw_forest_carbon_net_flux__Mg_CO2e),
              SUM(gfw_forest_carbon_gross_emissions__Mg_CO2e),
              SUM(gfw_forest_carbon_gross_removals__Mg_CO2e)
        FROM data
        WHERE umd_tree_cover_density_2000__threshold >= 30
        OR is__umd_tree_cover_gain = 'true'"""

result = d.query(sql=sql, geostore_id=gid, verbose=True)
result


KeyboardInterrupt: 